In [26]:
import os
import time
import copy
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
import torchvision.models as models
from torchvision import transforms

In [27]:
# Define variables
num_classes = 10
data_folder = 'data'
size = (500, 400)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [28]:
# Load data
# https://pytorch.org/docs/stable/torchvision/datasets.html#imagefolder

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

image_datasets = {x: torchvision.datasets.ImageFolder(os.path.join(data_folder, x), data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=8, shuffle=True, num_workers=4) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

In [35]:
class_names

['dino - 100px away',
 'dino - 150px away',
 'dino - 250px away',
 'dino - 75px away',
 'dino - 90px away',
 'pterodactyl - 100px away',
 'pterodactyl - 150px away',
 'pterodactyl - 250px away',
 'pterodactyl - 75px away',
 'pterodactyl - 90px away']

In [29]:
image_datasets['train'][0]

(tensor([[[2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489],
          [2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489],
          [2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489],
          ...,
          [2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489],
          [2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489],
          [2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489]],
 
         [[2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286],
          [2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286],
          [2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286],
          ...,
          [2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286],
          [2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286],
          [2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286]],
 
         [[2.6400, 2.6400, 2.6400,  ..., 2.6400, 2.6400, 2.6400],
          [2.6400, 2.6400, 2.6400,  ..., 2.6400, 2.6400, 2.6400],
          [2.6400, 2.6400, 2.6400,  ...,

In [30]:
# Create model
# https://pytorch.org/docs/stable/torchvision/models.html
# https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
model = models.resnet18(pretrained=True)

# Change number of output features
# model_ft = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model.fc = nn.Linear(num_ftrs, len(class_names))

model = model.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)


In [31]:
# Training code
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [32]:
# Let's train
model_ft = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=10)

Epoch 0/9
----------
train Loss: 2.2794 Acc: 0.1160
val Loss: 2.1904 Acc: 0.1333

Epoch 1/9
----------
train Loss: 1.9276 Acc: 0.3094
val Loss: 1.6030 Acc: 0.4667

Epoch 2/9
----------
train Loss: 1.4925 Acc: 0.5470
val Loss: 1.2152 Acc: 0.7333

Epoch 3/9
----------
train Loss: 1.1119 Acc: 0.7348
val Loss: 0.9468 Acc: 0.7000

Epoch 4/9
----------
train Loss: 0.9179 Acc: 0.8122
val Loss: 0.6972 Acc: 0.8667

Epoch 5/9
----------
train Loss: 0.6332 Acc: 0.9006
val Loss: 0.5749 Acc: 0.9000

Epoch 6/9
----------
train Loss: 0.4832 Acc: 0.9448
val Loss: 0.4655 Acc: 0.9000

Epoch 7/9
----------
train Loss: 0.4100 Acc: 0.9724
val Loss: 0.4232 Acc: 0.8667

Epoch 8/9
----------
train Loss: 0.3822 Acc: 0.9724
val Loss: 0.4044 Acc: 0.9000

Epoch 9/9
----------
train Loss: 0.3975 Acc: 0.9669
val Loss: 0.3921 Acc: 0.9000

Training complete in 25m 16s
Best val Acc: 0.900000


In [33]:
torch.save(model_ft.state_dict(),'test.pth')

In [34]:
!ls -larth

total 43M
-rw-rw-r-- 1 alexander alexander 3.2K Feb 22 17:42 gameV3.py
drwxrwxr-x 9 alexander alexander 4.0K Feb 22 17:57 ..
-rw-r--r-- 1 alexander alexander   23 Feb 22 17:58 README.md
drwxrwxr-x 3 alexander alexander 4.0K Feb 22 18:23 .idea
drwxrwxr-x 4 alexander alexander 4.0K Feb 22 18:44 data
-rw-r--r-- 1 alexander alexander 1.8K Feb 22 19:12 .gitignore
drwxr-xr-x 2 alexander alexander 4.0K Feb 22 19:18 .ipynb_checkpoints
-rw-r--r-- 1 alexander alexander 6.8K Feb 22 20:24 Predict.ipynb
drwxr-xr-x 8 alexander alexander 4.0K Feb 22 20:35 .git
-rw-r--r-- 1 alexander alexander  11K Feb 22 21:00 train_classifier.ipynb
drwxr-xr-x 6 alexander alexander 4.0K Feb 22 21:00 .
-rw-r--r-- 1 alexander alexander  43M Feb 22 21:02 test.pth
